In [6]:
import cv2
import keras
from keras import layers
import numpy as np
import os

In [7]:
%run read-image.ipynb
# %run prepare_data.ipynb

In [8]:
# train_dataset, valid_dataset, test_dataset, train_count, valid_count, test_count = generate_datasets()

In [9]:
from glob import glob
path = "/root/pythonprojects/deeplearning/covidDataset"
Train_dir = os.path.join(path,"Train")
covid_image_path = glob(os.path.join(Train_dir,"COVID", "*.png"))
non_covid_image_path = glob(os.path.join(Train_dir,"non-COVID", "*.png"))
Test_dir = os.path.join(path,"Test")
covid_image_path_test = glob(os.path.join(Test_dir,"COVID", "*.png"))
non_covid_image_path_test = glob(os.path.join(Test_dir,"non-COVID", "*.png"))
# print(len(covid_image_path))
# print(len(non_covid_image_path))
# print(len(covid_image_path_test))
# print(len(non_covid_image_path_test))

In [10]:
# img = shape(covid_image_path[100], flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=248, plot=True)
# print(img[0])
# print(img[1].shape)
# help(shape)
# print(type(img))
train = []
test = []
size = 128
for path in covid_image_path:
    img = shape(path, flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=size, plot=False)
    train.append(img)
for path in non_covid_image_path:
    img = shape(path, flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=size, plot=False)
    train.append(img)
for path in covid_image_path_test:
    img = shape(path, flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=size, plot=False)
    test.append(img)
for path in non_covid_image_path_test:
    img = shape(path, flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=size, plot=False)
    test.append(img)

    
train = np.array(train)
test = np.array(test)

In [11]:
# image = shape(covid_image_path[0], flag1 = cv2.IMREAD_GRAYSCALE, flag2=True, size=128, plot=True)
# import matplotlib.pyplot as plt
# plt.imshow(train[0], cmap='gray')
# plt.show()
# print(len(train[:]))
# print(train.shape)

In [12]:
# print(f"train -{train.shape}; test-{test.shape}")
# x_train = train.astype('float32') / 255.
# x_test = test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
# print(x_train.shape)
# print(x_test.shape)

## Autoencoder Model

In [13]:
# encoding_dims = 32

# input_img = keras.Input(shape = (784, ))

# encoded = layers.Dense(encoding_dims, activation="relu")(input_img)

# decoded = layers.Dense(784, activation="sigmoid")(encoded)

# autoencoder = keras.Model(input_img, decoded)

# encoder = keras.Model(input_img, encoded)

# encoded_input = keras.Input(shape=(encoding_dims, ))
# decoder_layer = autoencoder.layers[-1]
# decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

# autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [14]:
# autoencoder.fit(x_train, x_train,
#                 epochs=10,
#                 batch_size=32,
#                 shuffle=True,
#                 validation_data=(x_test, x_test))

In [15]:
# encoded_imgs = encoder.predict(x_test)
# decoded_imgs = decoder.predict(encoded_imgs)

In [16]:
# # plots
# n = 10  # How many digits we will display
# plt.figure(figsize=(20, 4))
# for i in range(n):
#     # Display original
#     ax = plt.subplot(2, n, i + 1)
#     plt.imshow(x_test[i].reshape(28, 28))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

#     # Display reconstruction
#     ax = plt.subplot(2, n, i + 1 + n)
#     plt.imshow(decoded_imgs[i].reshape(28, 28))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
# plt.show()

In [17]:
input_img = keras.Input(shape=(128, 128, 1))

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [20]:
x_train = np.asarray(train).astype('float32') / 255.
x_test = np.asarray(test).astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 128, 128, 1))
x_test = np.reshape(x_test, (len(x_test), 128, 128, 1))
print(f"train -{x_train.shape}; test-{x_test.shape}")

train -(2002, 128, 128, 1); test-(479, 128, 128, 1)


In [21]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/50


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr/local/lib/python3.8/dist-packages/keras/losses.py", line 141, in __call__
        losses = call_fn(y_true, y_pred)
    File "/usr/local/lib/python3.8/dist-packages/keras/losses.py", line 2